<a href="https://colab.research.google.com/github/srilamaiti/fall_2023_210_capstone_deepika_srila/blob/main/final_deliverable/model/train_val_test_file_generation%20/generate_train_val_test_files_5classes_recorded_mp4_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 41.1 MB/s eta 0:00:00


In [ ]:
import os
import random
import tqdm
import sys
import pathlib
import itertools
import collections
from pathlib import Path
from sklearn.utils import shuffle
import shutil
import glob
from glob import glob
import urllib.request
import subprocess
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Mounted at /content/drive


In [ ]:
LABEL_LIST = ['hello', 'thankyou', 'teacher', 'class', 'love']

label_info_dict = {}
for label in LABEL_LIST:
    label_info_dict[label] = {'gdrive_mp4_path_1' : os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'recorded_videos', f'{label}_mp4_videos'),
                              'gdrive_mp4_path_1_file_count' : len(glob(os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'recorded_videos', f'{label}_mp4_videos') + "/*.mp4")),
                              'gdrive_mp4_path_2' : os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'webscraped_videos', f'{label}_mp4_videos'),
                              'gdrive_mp4_path_2_file_count' : len(glob(os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'webscraped_videos', f'{label}_mp4_videos') + "/*.mp4"))
                             }
label_info_dict

{'hello': {'gdrive_mp4_path_1': '/content/drive/MyDrive/recorded_videos/hello_mp4_videos',
  'gdrive_mp4_path_1_file_count': 21,
  'gdrive_mp4_path_2': '/content/drive/MyDrive/webscraped_videos/hello_mp4_videos',
  'gdrive_mp4_path_2_file_count': 0},
 'thankyou': {'gdrive_mp4_path_1': '/content/drive/MyDrive/recorded_videos/thankyou_mp4_videos',
  'gdrive_mp4_path_1_file_count': 21,
  'gdrive_mp4_path_2': '/content/drive/MyDrive/webscraped_videos/thankyou_mp4_videos',
  'gdrive_mp4_path_2_file_count': 0},
 'teacher': {'gdrive_mp4_path_1': '/content/drive/MyDrive/recorded_videos/teacher_mp4_videos',
  'gdrive_mp4_path_1_file_count': 20,
  'gdrive_mp4_path_2': '/content/drive/MyDrive/webscraped_videos/teacher_mp4_videos',
  'gdrive_mp4_path_2_file_count': 0},
 'class': {'gdrive_mp4_path_1': '/content/drive/MyDrive/recorded_videos/class_mp4_videos',
  'gdrive_mp4_path_1_file_count': 21,
  'gdrive_mp4_path_2': '/content/drive/MyDrive/webscraped_videos/class_mp4_videos',
  'gdrive_mp4_path_

In [ ]:
def create_dir(dir_path):
    os.makedirs(dir_path, exist_ok=True)

def copy_file(file_name, dir_path):
    !cp {file} {dir_path}

def copy_set_of_files(file_list, dir_path):
    #print(len(file_list), dir_path)
    for file in file_list:
        #print(f"Copying {file} to {dir_path}")
        #copy_file(file_name = file, dir_path = dir_path)
        !cp {file} {dir_path}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")
    #print(os.listdir(dir_path))

def remove_files_from_dir(dir_path):
    for file in os.listdir(dir_path):
        qualified_file_name = os.path.join(dir_path, file)
        !rm -rf {qualified_file_name}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")

In [ ]:
random.seed(1234)
os.environ['PYTHONHASHSEED'] = '1234'

In [ ]:
SPLITS = (.6, .2, .2)
TOTAL_SAMPLES_PER_LABEL = 10

TRAIN_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[0])
VAL_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[1])
TEST_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[2])

print(f"TRAIN_RECS_PER_LABEL : {TRAIN_RECS_PER_LABEL}")
print(f"VAL_RECS_PER_LABEL : {VAL_RECS_PER_LABEL}")
print(f"TEST_RECS_PER_LABEL : {TEST_RECS_PER_LABEL}")

TRAIN_RECS_PER_LABEL : 6
VAL_RECS_PER_LABEL : 2
TEST_RECS_PER_LABEL : 2


In [ ]:
TRAIN_FILE_LIST = []
VAL_FILE_LIST = []
TEST_FILE_LIST = []

# Saving files in gdrive
#current_ts = datetime.today().isoformat().replace("-","").replace("T","").replace(":","").replace(".","")
TRAIN_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'train_files.txt')
VAL_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'val_files.txt')
TEST_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'test_files.txt')

train_file_pointer = open(TRAIN_FILE_NAME, 'w')
val_file_pointer = open(VAL_FILE_NAME, 'w')
test_file_pointer = open(TEST_FILE_NAME, 'w')

for label in LABEL_LIST:
    gdrive_path_1 = label_info_dict[label]['gdrive_mp4_path_1'] # recorded videos

    # test
    temp_test_files_1 = subprocess.getoutput(f"shuf -zn{TEST_RECS_PER_LABEL} -e {gdrive_path_1}/*.mp4 ").split("\n")
    test_files_1 = [e.split(" -> ")[0].replace("'","") for e in temp_test_files_1]
    test_files_1 = test_files_1[0].split("\x00")[:-1]
    test_files = test_files_1

    # train
    temp_train_files_1 = list(set([os.path.join(gdrive_path_1, file) for file in os.listdir(gdrive_path_1)]) - set(test_files_1))
    temp_train_files_1 = sorted(temp_train_files_1, key=lambda x: random.random())
    train_files_1 = random.sample(temp_train_files_1, TRAIN_RECS_PER_LABEL)
    train_files = train_files_1

    # val
    temp_val_files_1 = list(set([os.path.join(gdrive_path_1, file) for file in os.listdir(gdrive_path_1)]) - set(test_files_1) - set(train_files_1))
    temp_val_files_1 = sorted(temp_val_files_1, key=lambda x: random.random())
    val_files_1 = random.sample(temp_val_files_1, VAL_RECS_PER_LABEL)
    val_files = val_files_1

    [train_file_pointer.writelines(e + "\n") for e in train_files]
    [val_file_pointer.writelines(e + "\n") for e in val_files]
    [test_file_pointer.writelines(e + "\n") for e in test_files]

# File closing
train_file_pointer.close()
val_file_pointer.close()
test_file_pointer.close()

In [ ]:
!cat {TRAIN_FILE_NAME}|wc -l

30


In [ ]:
!cat {TEST_FILE_NAME}|wc -l

10


In [ ]:
!cat {VAL_FILE_NAME}|wc -l

10


In [ ]:
TRAIN_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'train')
VAL_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'val')
TEST_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '5classes_train_val_test_recorded_mp4_videos', 'test')

for label in LABEL_LIST:
    train_dir_name = os.path.join(TRAIN_FILE_PATH, label)
    create_dir(train_dir_name)

    val_dir_name = os.path.join(VAL_FILE_PATH, label)
    create_dir(val_dir_name)

    test_dir_name = os.path.join(TEST_FILE_PATH, label)
    create_dir(test_dir_name)

In [ ]:
def copy_file_to_folder(file_name, dir_path):
    fp = open(file_name, "r")
    for file in fp:
        file = file.strip()
        label = file.split("/")[-2].split("_")[0]
        base_file_name = os.path.basename(file)
        dest_path = os.path.join(dir_path, label)
        shutil.copy(file, dest_path)
        #!ls -ltrSh {dir_path}/{label}/*.avi | wc -l

In [ ]:
for label in LABEL_LIST:
    !rm -rf {TRAIN_FILE_PATH}/{label}/*.mp4
    !rm -rf {VAL_FILE_PATH}/{label}/*.mp4
    !rm -rf {TEST_FILE_PATH}/{label}/*.mp4

copy_file_to_folder(file_name = TRAIN_FILE_NAME, dir_path = TRAIN_FILE_PATH)
copy_file_to_folder(file_name = VAL_FILE_NAME, dir_path = VAL_FILE_PATH)
copy_file_to_folder(file_name = TEST_FILE_NAME, dir_path = TEST_FILE_PATH)

for label in LABEL_LIST:
    !ls -ltrSh {TRAIN_FILE_PATH}/{label}/*.mp4 | wc -l
    !ls -ltrSh {VAL_FILE_PATH}/{label}/*.mp4 | wc -l
    !ls -ltrSh {TEST_FILE_PATH}/{label}/*.mp4 | wc -l

6
2
2
6
2
2
6
2
2
6
2
2
6
2
2


In [ ]:
print("Mediapipe initiation")
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

Mediapipe initiation
